## IMPORT MODULE

In [1]:
# Modules for web scraping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
# URL encoding
from requests.utils import requote_uri
# Module for data manipulation
import pandas as pd
# Module for regular expression
import re
# Module for file management
import os
# Module for timing
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

**Geckodriver path**

In [2]:
# Options for Chrome windows
options = webdriver.FirefoxOptions()
options.add_argument("--start-maximized")
options.add_argument('window-size=2560,1440')
options.add_argument('disable-notifications')
options.add_argument('--disable-infobars')
options.add_argument('start-maximized')

# supaya gak muncul mozilla nya
# options.add_argument('--headless')

In [12]:
DRIVER_PATH = 'geckodriver'
driver = webdriver.Firefox(executable_path = DRIVER_PATH,options=options)

**URL and Query**

In [4]:
url_main = 'https://shopee.co.id/Pakaian-Wanita-cat.32'

**Access website**

In [13]:
driver.get(url_main)

# WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.shopee-search-item-result__items')))
# driver.implicitly_wait(20)


In [6]:
def scroll_window():
    scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')
    for iter in range(0,scroll_height,200):
        driver.execute_script('window.scrollTo(0, {});'.format(iter))
        time.sleep(1.5)
    pass

### PER ELEMENT VALUE

In [7]:
# scroll_window()

# elements = WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[4]/div[2]/div/div[2]//a')))
page_element_product = driver.find_element_by_css_selector(".shopee-search-item-result")

current_page = int(page_element_product.find_element_by_css_selector(".shopee-mini-page-controller__current").text)
page_total = int(page_element_product.find_element_by_css_selector(".shopee-mini-page-controller__total").text)
next_page_btn = page_element_product.find_element_by_xpath('//*[@id="main"]/div/div[2]/div[2]/div[4]/div[2]/div/div[1]/div[2]/button[2]')
# '//*[@id="main"]/div/div[2]/div[2]/div[4]/div[2]/div/div[1]/div[2]/button[2]'

products_list = []

for i in range(page_total):
    scroll_window()
    elements = WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[4]/div[2]/div/div[2]//a')))
    for element in elements:
        #### get href
        href = element.get_attribute('href')
        name = element.find_element_by_xpath('//*[@class="_1NoI8_ _16BAGk"]').text
        products_list.append({
            'product_name': name,
            'product_link': href
        })
    next_page_btn = page_element_product.find_element_by_xpath('//*[@id="main"]/div/div[2]/div[2]/div[4]/div[2]/div/div[1]/div[2]/button[2]')
    next_page_btn.click()
    WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="main"]/div/div[2]/div[2]/div[4]/div[2]/div')))



KeyboardInterrupt: 

In [11]:
products_list
df_product = pd.DataFrame.from_dict(products_list)
df_product.to_excel("daftar_link_product_to_follow.xlsx")